In [1]:
print("cdsc")

cdsc


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine, MetaData, Table
import psycopg2

In [4]:
# подгружаем .env
load_dotenv()

True

In [11]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')


YCAJE26tCTzKqvgfV8G9aEd-k


In [6]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [7]:
import boto3
import os

class Config:
    AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

    S3_SERVICE_NAME = 's3'
    S3_ENDPOINT_URL = 'https://storage.yandexcloud.net'


def get_session():
    session = boto3.session.Session()

    return session.client(
        service_name=Config.S3_SERVICE_NAME,
        endpoint_url=Config.S3_ENDPOINT_URL,
        aws_access_key_id=Config.AWS_ACCESS_KEY_ID,
        aws_secret_access_key=Config.AWS_SECRET_ACCESS_KEY
    )

bucket_name = os.getenv("S3_BUCKET_NAME")

s3 = get_session()

if s3.list_objects(Bucket=bucket_name).get('Contents'):
    for key in s3.list_objects(Bucket=bucket_name)['Contents']:
        print(key['Key'])

10/d923c249e6db43458920a7188830f359/artifacts/dataframe/columns.txt
10/d923c249e6db43458920a7188830f359/artifacts/dataframe/users_churn.csv
11/ce28e84977d84ce9b5506339f3a6b21e/artifacts/dataframe/columns.txt
11/ce28e84977d84ce9b5506339f3a6b21e/artifacts/dataframe/users_churn.csv
13/ba5cb3a55d7a48039dee4c6640a6e275/artifacts/dataframe/columns.txt
13/ba5cb3a55d7a48039dee4c6640a6e275/artifacts/dataframe/users_churn.csv
15/a0f2102517d542f9af441608707430a6/artifacts/models/MLmodel
15/a0f2102517d542f9af441608707430a6/artifacts/models/conda.yaml
15/a0f2102517d542f9af441608707430a6/artifacts/models/input_example.json
15/a0f2102517d542f9af441608707430a6/artifacts/models/model.pkl
15/a0f2102517d542f9af441608707430a6/artifacts/models/python_env.yaml
15/a0f2102517d542f9af441608707430a6/artifacts/models/requirements.txt
15/b8de7a79924a4684a186b3618ed1013e/artifacts/models/MLmodel
15/b8de7a79924a4684a186b3618ed1013e/artifacts/models/conda.yaml
15/b8de7a79924a4684a186b3618ed1013e/artifacts/models/inp

In [8]:
import os
import boto3
from botocore.exceptions import ClientError

# Конфигурация (используйте переменные окружения для секретов!)
S3_ENDPOINT = "https://storage.yandexcloud.net"
S3_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID") # Замените на переменные окружения!
S3_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("S3_BUCKET_NAME")
FILE_KEY = "data/initial_data.csv"
LOCAL_PATH = "./initial_data.csv"  # Куда сохранить файл

# Инициализация клиента
s3 = boto3.client(
    "s3",
    endpoint_url=S3_ENDPOINT,
    aws_access_key_id=S3_ACCESS_KEY,
    aws_secret_access_key=S3_SECRET_KEY
)

try:
    # Скачивание файла
    s3.download_file(
        Bucket=BUCKET_NAME,
        Key=FILE_KEY,
        Filename=LOCAL_PATH
    )
    print(f"Файл {FILE_KEY} успешно скачан в {LOCAL_PATH}")

except ClientError as e:
    print(f"Ошибка доступа к S3: {e}")
except Exception as e:
    print(f"Общая ошибка: {e}")

Файл data/initial_data.csv успешно скачан в ./initial_data.csv


In [9]:
import os
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Загрузка данных
df = pd.read_csv("./initial_data.csv")

# Удаление ненужных столбцов (например, building_id)
data = df.drop(columns=["building_id"])

# Разделение на признаки и целевую переменную
X = data.drop(columns=["is_apartment"])
y = data["is_apartment"].astype(int)  # Преобразование boolean в 0/1

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Определение категориальных и числовых признаков
categorical_features = ["building_type_int"]
numeric_features = [col for col in X.columns if col not in categorical_features]

# Препроцессинг
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(), categorical_features),
    ]
)

# Пайплайн
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", RandomForestClassifier(random_state=42, class_weight="balanced")),
    ]
)
pipeline.fit(X_train, y_train)
from sklearn.metrics import (
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
)

# Предсказания
y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

# Метрики
metrics = {
    "roc_auc": roc_auc_score(y_test, y_pred_proba),
    "precision": precision_score(y_test, y_pred),
    "recall": recall_score(y_test, y_pred),
    "f1": f1_score(y_test, y_pred),
    "accuracy": accuracy_score(y_test, y_pred),
}

print("Метрики новой модели:")
for name, value in metrics.items():
    print(f"{name}: {value:.4f}")

/home/mle-user/.venv_project_name/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


Метрики новой модели:
roc_auc: 0.9428
precision: 0.7200
recall: 0.4417
f1: 0.5475
accuracy: 0.9944


In [24]:
import os
import mlflow
import pandas as pd
from mlflow.models.signature import infer_signature
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score
from mlflow.pyfunc import load_model  # Правильный импорт

# Исправление опечатки в названии переменной
EXPERIMENT_NAME = "churn_denis_putov_www"  
RUN_NAME = "model_222registry"
REGISTRY_MODEL_NAME = "churn_model_den"

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# Настройки для Yandex Cloud
os.environ.update({
    "MLFLOW_S3_ENDPOINT_URL": "https://storage.yandexcloud.net",
    "AWS_ACCESS_KEY_ID": os.getenv("AWS_ACCESS_KEY_ID"),  # Убедитесь, что переменные окружения установлены
    "AWS_SECRET_ACCESS_KEY": os.getenv("AWS_SECRET_ACCESS_KEY"),
})

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")


# Обучение модели CatBoost (удален дублирующий код с LogisticRegression)
model = CatBoostClassifier(iterations=1000, random_state=42)
model.fit(X_train, y_train)

# Генерация предсказаний и вычисление метрик
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

metrics = {
    "roc_auc": roc_auc_score(y_test, y_proba),
    "precision": precision_score(y_test, y_pred),
    "recall": recall_score(y_test, y_pred),
    "f1": f1_score(y_test, y_pred),
    "accuracy": accuracy_score(y_test, y_pred)
}

# Получение эксперимента
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if not experiment:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

# Логирование в одном контексте запуска
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # Логирование параметров модели
    mlflow.log_params(model.get_params())
    
    # Логирование метрик
    mlflow.log_metrics(metrics)
    

    # Подготовка метаданных модели
    pip_requirements = "../requirements.txt"  # Пример зависимостей # Пример из обучающих данных
    signature = mlflow.models.infer_signature(X_test, prediction)  # Исправлена опечатка
    input_example = X_test[:10]
    metadata = {'model_type': 'monthly'}
    # Логирование CatBoost модели
    model_info = mlflow.catboost.log_model(
        cb_model=model,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        pip_requirements=pip_requirements,
        signature=signature,
        input_example=input_example,
        metadata={'model_type': 'monthly'}
    )

loaded_model = load_model(model_info.model_uri)  # Теперь работает
predictions = loaded_model.predict(X_test)
print(predictions[:10])

Learning rate set to 0.068699
0:	learn: 0.5293107	total: 138ms	remaining: 2m 17s
1:	learn: 0.4011504	total: 198ms	remaining: 1m 38s
2:	learn: 0.3147799	total: 242ms	remaining: 1m 20s
3:	learn: 0.2395663	total: 293ms	remaining: 1m 13s
4:	learn: 0.1913780	total: 349ms	remaining: 1m 9s
5:	learn: 0.1545494	total: 413ms	remaining: 1m 8s
6:	learn: 0.1247099	total: 462ms	remaining: 1m 5s
7:	learn: 0.1040368	total: 512ms	remaining: 1m 3s
8:	learn: 0.0892421	total: 558ms	remaining: 1m 1s
9:	learn: 0.0756764	total: 622ms	remaining: 1m 1s
10:	learn: 0.0657220	total: 683ms	remaining: 1m 1s
11:	learn: 0.0588466	total: 794ms	remaining: 1m 5s
12:	learn: 0.0533696	total: 849ms	remaining: 1m 4s
13:	learn: 0.0492136	total: 906ms	remaining: 1m 3s
14:	learn: 0.0454169	total: 954ms	remaining: 1m 2s
15:	learn: 0.0423916	total: 1.01s	remaining: 1m 1s
16:	learn: 0.0400482	total: 1.08s	remaining: 1m 2s
17:	learn: 0.0381656	total: 1.15s	remaining: 1m 2s
18:	learn: 0.0369620	total: 1.2s	remaining: 1m 2s
19:	lear

/home/mle-user/.venv_project_name/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
Registered model 'churn_model_den' already exists. Creating a new version of this model...
2025/03/27 10:30:47 INFO mlflow.tracking._model_registry.client: Waitin

2025/03/27 10:30:58 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - aiobotocore (current: uninstalled, required: aiobotocore==2.11.1)
 - aiohttp (current: uninstalled, required: aiohttp==3.9.3)
 - aiohttp-retry (current: uninstalled, required: aiohttp-retry==2.8.3)
 - aioitertools (current: uninstalled, required: aioitertools==0.11.0)
 - aiosignal (current: uninstalled, required: aiosignal==1.3.1)
 - amqp (current: uninstalled, required: amqp==5.2.0)
 - annotated-types (current: 0.7.0, required: annotated-types==0.6.0)
 - antlr4-python3-runtime (current: uninstalled, required: antlr4-python3-runtime==4.9.3)
 - appdirs (current: uninstalled, required: appdirs==1.4.4)
 - async-timeout (current: uninstalled, required: async-timeout==4.0.3)
 - asyncssh (current: uninstalled, required: asyncssh==2.14.2)
 - atpublic (current: uninstalled, required: atpublic==4.0)
 - attrs (current: 25.3.0, required: attrs==23.2.0)

[0 0 0 0 0 0 0 0 0 0]


In [26]:
import os

import mlflow


EXPERIMENT_NAME = "churn_denis_putov_www"  
RUN_NAME = "model_222registry"
REGISTRY_MODEL_NAME = "churn_model_den"


TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

os.environ.update({
    "MLFLOW_S3_ENDPOINT_URL": "https://storage.yandexcloud.net",
    "AWS_ACCESS_KEY_ID": os.getenv("AWS_ACCESS_KEY_ID"), 
    "AWS_SECRET_ACCESS_KEY": os.getenv("AWS_SECRET_ACCESS_KEY"),
})

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")


pip_requirements = "../requirements.txt"  
signature = mlflow.models.infer_signature(X_test, prediction)  
input_example = X_test[:10]
metadata = {'model_type': 'monthly'}
metrics = {
    "roc_auc": roc_auc_score(y_test, y_proba),
    "precision": precision_score(y_test, y_pred),
    "recall": recall_score(y_test, y_pred),
    "f1": f1_score(y_test, y_pred),
    "accuracy": accuracy_score(y_test, y_pred)
}

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    mlflow.log_params(model.get_params())
    mlflow.log_metrics(metrics)
        
    model_info = mlflow.catboost.log_model(
    await_registration_for=60,
    cb_model=model,
    artifact_path="models",
    registered_model_name=REGISTRY_MODEL_NAME,
    pip_requirements=pip_requirements,
    signature=signature,
    input_example=input_example,
    metadata={'model_type': 'monthly'}
    )

/home/mle-user/.venv_project_name/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
Registered model 'churn_model_den' already exists. Creating a new version of this model...
2025/03/27 10:41:32 INFO mlflow.tracking._model_registry.client: Waitin

In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, log_loss
import os
import os
import joblib
import mlflow
# заведите словарь со всеми метриками
import os

import mlflow


EXPERIMENT_NAME = "churn_denis_putov"
RUN_NAME = "model_1_registry"
REGISTRY_MODEL_NAME = "churn_model_denis_putov_111"


os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")

# ваш код здесь


pip_requirements = "../requirements.txt"
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]
metadata = {'model_type': 'monthly'}

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("preprocessor", "StandardScaler + OneHotEncoder")
    run_id = run.info.run_id
    # ваш код здесь
    model_info = mlflow.catboost.log_model(
        await_registration_for=60,
        input_example=input_example,
        cb_model=model,
        pip_requirements=pip_requirements,
        signature=signature,
        metadata=metadata,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
    ) 
    
loaded_model = load_model(model_uri=model_info.model_uri)
model_predictions = loaded_model.predict(X_test)

assert model_predictions.dtype == int

print(model_predictions[:10])

In [ ]:
import mlflow
import mlflow.sklearn

# Настройка MLflow
mlflow.set_tracking_uri("http://localhost:5000")  # Укажите ваш tracking server
mlflow.set_experiment("Apartment Classification")

with mlflow.start_run():
    # Логирование гиперпараметров
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("preprocessor", "StandardScaler + OneHotEncoder")
    
    # Логирование метрик
    for name, value in metrics.items():
        mlflow.log_metric(name, value)
    
    # Логирование модели
    mlflow.sklearn.log_model(pipeline, "model", registered_model_name="ApartmentClassifier")

In [29]:
from mlflow import MlflowClient

# Инициализация клиента
client = MlflowClient()

# Получаем информацию о текущей модели (перед переименованием)
original_model_name = "churn_model_den"
model_details = client.get_registered_model(original_model_name)
print(f"Model info:\n{model_details}")

# Получаем все версии модели и сортируем по убыванию версии
model_versions = client.search_model_versions(f"name='{original_model_name}'")
sorted_versions = sorted(model_versions, key=lambda x: int(x.version), reverse=True)

# Проверяем, что есть как минимум две версии
if len(sorted_versions) < 2:
    raise ValueError("Для выполнения задания требуется как минимум две версии модели")

# Информация о последней и предпоследней версиях
model_name_1 = original_model_name
model_version_1 = sorted_versions[0].version
model_stage_1 = sorted_versions[0].current_stage

model_name_2 = original_model_name
model_version_2 = sorted_versions[1].version
model_stage_2 = sorted_versions[1].current_stage

print(f"\nТекущий stage модели 1 (v{model_version_1}): {model_stage_1}")
print(f"Текущий stage модели 2 (v{model_version_2}): {model_stage_2}")

# Меняем статусы
client.transition_model_version_stage(
    name=original_model_name,
    version=model_version_1,
    stage="Production"
)

client.transition_model_version_stage(
    name=original_model_name,
    version=model_version_2,
    stage="Staging"
)

# Переименовываем модель
new_model_name = f"{original_model_name}_b2c"
client.rename_registered_model(
    name=original_model_name,
    new_name=new_model_name
)

print(f"\nМодель успешно переименована в: {new_model_name}")

Model info:
 []


IndexError: list index out of range